In [35]:
## get LAD names from ONS WFS

import geopandas as gpd
import requests
import geojson
import time

# Specify the URL for the backend.
url = 'https://dservices1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/services/Local_Authority_Districts_December_2023_Boundaries_UK_BFC/WFSServer'

# Specify parameters (read data in geojson format).
params = dict(
    service='WFS',
    version='2.0.0',
    request='GetFeature',
    typeName='Local_Authority_Districts_December_2023_Boundaries_UK_BFC:LAD_DEC_2023_UK_BFC',
    outputFormat='GEOJSON'
)

# Define a function to fetch data with retry mechanism
def fetch_data(url, params, retries=3, timeout=60):
    for i in range(retries):
        try:
            response = requests.get(url, params=params, timeout=timeout)
            if response.status_code == 200:
                return response
            else:
                print(f"Failed to retrieve data: {response.status_code} - {response.reason}")
        except requests.exceptions.RequestException as e:
            print(f"Attempt {i+1} failed: {e}")
            time.sleep(5)  # wait before retrying
    return None

# Fetch data from WFS using requests
response = fetch_data(url, params)

# Check if the request was successful
if response and response.status_code == 200:
    # Load the response content as GeoJSON
    geojson_data = geojson.loads(response.content)
    
    # Create a GeoDataFrame from the GeoJSON data and set the coordinate reference system (CRS)
    gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])
    
    # Print information about the GeoDataFrame
    print(gdf.info())
    
    # Optionally, save the GeoDataFrame to a file
    gdf.to_file("Local_Authority_Districts.geojson", driver='GeoJSON')
    
    # Display the first few rows of the GeoDataFrame
    print(gdf.head())
else:
    print("Failed to retrieve data after multiple attempts.")


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   geometry       361 non-null    geometry
 1   GmlID          361 non-null    object  
 2   FID            361 non-null    int64   
 3   LAD23CD        361 non-null    object  
 4   LAD23NM        361 non-null    object  
 5   LAD23NMW       361 non-null    object  
 6   BNG_E          361 non-null    int64   
 7   BNG_N          361 non-null    int64   
 8   LONG           361 non-null    float64 
 9   LAT            361 non-null    float64 
 10  Shape__Area    361 non-null    float64 
 11  Shape__Length  361 non-null    float64 
 12  GlobalID       361 non-null    object  
dtypes: float64(4), geometry(1), int64(3), object(5)
memory usage: 36.8+ KB
None
                                            geometry                  GmlID  \
0  MULTIPOLYGON (((-1.22470 54.62611, -1.22467 54...

In [38]:
output_file = r'C:\Users\b8008458\OneDrive - Newcastle University\2022 to 2023\PhD\ltnDetection\LTN-Detection\data\output.txt'

with open(output_file, 'w') as file:
    for value in gdf['LAD23NM']:
        file.write(value + '\n')
